In [19]:
import numpy as np
import pandas as pd
from collections import Counter
import statistics as st
import math
import time


# Banco de Dados

Foi realizada uma busca de banco de dados no site [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php), pois é um repositório com diversos bancos voltados para o tema de aprendizado de máquina.

Foram cogitados vários bancos diferentes, como um voltado para a associação das bandeiras dos países com a religião dominante e também outro que informava os diferentes tipos de vidro com suas características, porém o escolhido para o treinamento e teste da Árvore de Decisão e da Rede Neural foi a que armazena o tipo de vinho e seus 13 constituintes.

O banco de dados de vinho possui 178 amostras e 13 atributos, além da coluna Vinho que possui a classificação de cada amostra. Os dados foram colhidos a partir de análises químicas de vinhos produzidos na Itália, mas foram coletados de três diferentes vinhedos.

Como esta base foi divida em duas partes, onde 70% para treinamento e 30% para teste, duas bases distintas foram geradas a partir da original, onde cada uma possui 125 e 53 linhas, respectivamente.

Como é possível verificar na tabela abaixo, a maioria dos atributos são valores float, pois tratam-se dos valores da composição de vinhos, com exceção dos atributos Magnesium e Prolin. Abaixo está a representação de cada atributo da base:

* **Vinho**: Os diferentes tipos de classificação de vinho, variando entre 1 e 3;
* **Alcohol**: A quantidade de Álcool no vinho, variando entre 11.03 e 14.83;
* **Malic Acid**:  A quantidade de Ácido metálico no vinho, variando entre 0.74 e 5.8;
* **Ash**: A quantidade de cinzas no vinho, variando entre 1.36 e 3.23;
* **Alcalinity of Ash**:  A  Alcalinidade  das  cinzas  no  vinho,  variando  entre 10.6 e 30;
* **Magnesium**: A quantidade de Magnésio no vinho, variando entre 70 e 162;
* **Total phenols**: A quantidade total de fenóis no vinho, variando entre 0.98 e 3.88;
* **Flavanoids**: A quantidade de flavonoides no vinho, variando entre 0.34 e 5.08;
* **Nonflavanoid phenols**: A quantidade de fenóis não flavonoides no vinho,variando entre 0.13 e 0.66;
* **Proanthocyanins**:  A quantidade de Ácido metálico no vinho, variando entre 0.41 e 3.58;
* **Color intensity**:  O valor da intensidade da cor do vinho,  variando entre 1.28 e 13;
* **Hue**: O valor da cor do vinho, variando entre 0.48 e 1.71;
* **OD280/OD315 of diluted wines**: A  quantidade  da  concentração de proteína no vinho, variando entre 1.27 e 4;
* **Prolin**: A quantidade de proline no vinho, variando entre 278 e 1680.

In [20]:
bd_vinho = pd.read_csv('banco_vinho.csv')
bd_vinho.head()

,Vinho,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,Diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# Ferramentas

Algumas funções que serão usadas tanto no tratamento de dados quanto na execução final do algoritmo.

In [21]:
# Aqui obtemos a base de treino, onde a proporção escolhida foi de 70%.
def retorna_treino(base):
    treino = base.sample(frac=0.7)
    return treino

# A base de teste é obtida retirando a base de treino do Dataframe.
def retorna_teste(base, base_treino):
    teste = base.drop(base_treino.index)
    return teste

# Nesta função, calculamos a sensibilidade, confiabilidades,
# dentre outros a partir da matriz de confusão.
def calcula_resultados(matriz, verbose=False):
    
    num_classes = len(matriz)

    sensibilidade = np.zeros([num_classes])
    especificidade = np.zeros([num_classes])
    confiabilidade_positiva = np.zeros([num_classes])
    confiabilidade_negativa = np.zeros([num_classes])

    tp = np.zeros([num_classes])
    tn = np.zeros([num_classes])
    fn = np.zeros([num_classes])
    fp = np.zeros([num_classes])

    acertos = 0
    acuracia = 0
    total = 0

    for classe in range(num_classes):
        for linha in range(num_classes):
            for coluna in range(num_classes):
                if classe == linha == coluna:
                    tp[classe] += matriz[linha][coluna]
                elif classe != linha == coluna:
                    tn[classe] += matriz[linha][coluna]
                elif classe == linha != coluna:
                    fn[classe] += matriz[linha][coluna]
                elif classe == coluna != linha:
                    fp[classe] += matriz[linha][coluna]

    if verbose:
        print('TP =', tp)
        print('TN =', tn)
        print('FN =', fn)
        print('FP =', fp)

    for linha in range(num_classes):
        for coluna in range(num_classes):
            if linha == coluna:
                acertos += matriz[linha][coluna]
            total += matriz[linha][coluna]

    acuracia = (acertos*100)/total

    for classe in range(num_classes):
        sensibilidade[classe] = tp[classe]/(tp[classe] + fn[classe])
        especificidade[classe] = tn[classe]/(tn[classe] + fp[classe])
        confiabilidade_positiva[classe] = tp[classe]/(tp[classe] + fp[classe])
        confiabilidade_negativa[classe] = tn[classe]/(tn[classe] + fn[classe])
        
        if tp[classe] + fn[classe] == 0:
            sensibilidade[classe] = 0
        if tn[classe] + fp[classe] == 0:
            especificidade[classe] = 0
        if tp[classe] + fp[classe] == 0:
            confiabilidade_positiva[classe] = 0
        if tn[classe] + fn[classe] == 0:
            confiabilidade_negativa[classe] = 0
    
    # Se quisermos ver todos os detalhes da execução, podemos escolher este
    # atributo verbose como True.
        if verbose:
            print('----------- Classe %d -----------' %(classe+1))
            print('Sensibilidade: ', sensibilidade[classe])
            print('Especificidade: ', especificidade[classe])
            print('Confiabilidade Positiva: ', confiabilidade_positiva[classe])
            print('Confiabilidade Negativa: ', confiabilidade_negativa[classe])
    if verbose:
        print('Media da Sensibilidade: ', np.mean(sensibilidade))
        print('Media da Especificidade: ', np.mean(especificidade))
        print('Media da Confiabilidade Positiva: ', np.mean(confiabilidade_positiva))
        print('Media da Confiabilidade Negativa: ', np.mean(confiabilidade_negativa))

    if verbose:
        print('Acurácia:', acuracia, end='\n')
    
    # Por fim, retornamos a acurácia da execução.
    return acuracia

# Tratamento do Banco de Dados

Começamos tratando o banco de dados, de modo a obter a base de dados original dividida entre base de treino e base de teste, com a proporção de 70/30. A função também nos retorna os tipos de saídas possíveis da coluna alvo, as classes que iremos prever.

In [22]:
# Nesta função, tratamos a base de dados, encapsulando a chamada
# de outras funções, a fim de retornar os dados separados.
def tratar_bd(banco, coluna):
    tipos_saidas = banco[coluna].unique()

    base_treino = np.array([])
    base_teste = np.array([])

    tamanho_treino = 0
    tamanho_teste = 0
    
    for classe in tipos_saidas:
        banco_auxiliar = banco.query('%s==%d' % (coluna, classe))
        treino_auxiliar = retorna_treino(banco_auxiliar)

        for linha in treino_auxiliar.values:
            base_treino = np.concatenate((base_treino, linha), axis=0)

        teste_auxiliar = retorna_teste(banco_auxiliar, treino_auxiliar)

        for linha in teste_auxiliar.values:
            base_teste = np.concatenate((base_teste, linha), axis=0)

        tamanho_treino += len(treino_auxiliar)
        tamanho_teste += len(teste_auxiliar)
        
    num_colunas = len(banco.columns)

    base_treino = np.reshape(base_treino, (tamanho_treino, num_colunas))
    base_teste = np.reshape(base_teste, (tamanho_teste, num_colunas))
    base_treino = pd.DataFrame(base_treino, columns=banco.columns)
    base_teste = pd.DataFrame(base_teste, columns=banco.columns)

    # Ao fim do tratamento, retornamos a base de treino e teste,
    # assim como as classes possíveis da coluna alvo.
    return base_treino, base_teste, tipos_saidas

# Classes de Nós

Construímos duas classes de Nós, uma sendo um Nó padrão, possuindo diversps atributos, tendo como destaque o atributo da pergunta, que será utilizada para dividir a base de dados em dois novos nós. Este tipo de nó é responsável por tomar a decisão no momento de classificar algum dado.

Já o **Nó Folha** fica responsável por dar de fato a classificação ao dado, assim que percorrer a árvore inteira, ele recebe o banco de dados e o alvo, e como será mostrando mais a frente, sua classe é definida pela moda do subconjunto de dados presente no nó.

In [23]:
class No(object):
    def __init__(self, atributo=None, entropia=None, pergunta=None, filho_esquerdo=None, filho_direito=None):
        self.atributo = atributo
        self.entropia = entropia
        self.pergunta = pergunta
        self.filho_esquerdo = filho_esquerdo
        self.filho_direito = filho_direito
    
    def __repr__(self):
        return '{} - {} - {}'.format(self.atributo, self.entropia, self.pergunta)

# Apesar de não ser necessário para o nó ou a classificação em si, o Nó Folha possui atributos de filhos,
# que são utilizados pela função responsável por calcular a altura da árvore.
class Folha(object):
    def __init__(self, banco, alvo):
        self.banco = banco
        self.alvo = alvo
        self.filho_esquerdo = None
        self.filho_direito = None
        self.classe = st.mode(self.banco[self.alvo])


# Classe Pergunta

A classe pergunta é a responsável por dividir o banco de dados a cada nó de decisão, ela armazena o atributo e o valor feitos na divisão, como por exemplo "Color Intensity >= 4". Ela é capaz de receber até mesmo classes categóricas, embora o banco utilizado no trabalho tenha somente valores númericos.

In [24]:
class Pergunta(object):

    def __init__(self, coluna, valor):
        self.coluna = coluna
        self.valor = valor

    def is_numeric(self, valor):
        return isinstance(valor, int) or isinstance(valor, float)

    # Esta função nos retorna o valor verdade da pergunta do nó de decisão.
    def verifica(self, exemplo):
        valor = exemplo[self.coluna]

        if self.is_numeric(valor):
            return valor >= self.valor
        else:
            return valor == self.valor

    def __repr__(self):
        condicao = "=="
        if self.is_numeric(self.valor):
            condicao= ">="
        return "%s %s %s?" % (self.coluna, condicao, str(self.valor))


# Árvore de Decisão

A Árvore de decisão utilizada neste trabalho foi baseada no algoritmo C 4.5, que é uma melhoria do algoritmo ID3. A escolha desse algoritmo foi motivada pelos recursos fornecidos, como o tratamento de valores discretos e contínuos, assim permitindo a mineração em diferentes tipos de bancos de dados.
Semelhante ao algoritmo C 4.5, a entropia é utilizada juntamente com o ganho de informação normalizado para realizar a partição dos dados 

Inicialmente é realizado o cálculo da entropia utilizando toda a base de treino, tendo como parâmetro a coluna alvo, **Vinho**. Em seguida são feitas partições na base considerando cada atributo que compõe o vinho, com o objetivo de encontrar a melhor divisão possível.

Para isto, os valores dos atributos são coletados, uma pergunta é criada utilizando o arredondamento de cada valor, por exemplo: "valor >= X ?", e tal pergunta é utilizada para dividir o banco em duas partes, onde tais partes são o filho esquerdo, caso os valores retornem falso com a pergunta, e o filho direito, caso o valores retornem verdadeira com a pergunta, de um nó de decisão.

Em seguida, é feito o cálculo de ganho de informação normalizado de cada filho de todos os valores de todos os atributos.

Uma vez que e o melhor ganho de informação é obtido, o particionamento é realizado, e tal processo é repetido até que todos os ramos tenham uma folha no final. E para que uma folha se forme, um filho de algum nó de decisão deve possuir no mínimo uma classe com mais de 80% de dominância, não importando a quantidade de classes que o subconjunto possui.

In [25]:
# Classe englobando a estrutura da Árvore de Decisão e todas as suas funções
class ArvoreDecisao(object):

    # A classe recebe o banco de dados e o alvo
    def __init__(self, banco_de_dados, coluna_alvo):
        
        self.banco_de_dados = banco_de_dados

        self.coluna_alvo = coluna_alvo
        self.dados_alvo = self.banco_de_dados[coluna_alvo]

        self.n_linhas = banco_de_dados.shape[0]
        self.n_colunas = banco_de_dados.shape[1]
        self.colunas = banco_de_dados.columns

        self.raiz = None
        
        self.cria_arvore()

    def __repr__(self):
        return 'Linhas:{}\nColunas:{}'.format(self.n_linhas, self.n_colunas)

    # Função que mede a altura da árvore, chama uma função privada
    def altura(self):
        return self._altura_recursiva(self.raiz, 0)

    # Aqui a árvore é percorrida de forma recursiva, e o valor da maior altura encontrada é retornada.
    def _altura_recursiva(self, no_atual, altura_atual):
        if not no_atual:
            return altura_atual
        altura_esquerdo = self._altura_recursiva(no_atual.filho_esquerdo, altura_atual + 1)
        altura_direito = self._altura_recursiva(no_atual.filho_direito, altura_atual + 1)
        return max(altura_esquerdo, altura_direito)

    # Função responsável pela criação da árvore, chamada quando o objeto é instanciado.
    def cria_arvore(self):
        self.raiz = self._cria_arvore_recursiva(self.banco_de_dados)

    # Cria a árvore de forma recursiva, recebendo apenas o banco de dados.
    def _cria_arvore_recursiva(self, banco):

        # O primeiro nó é criado, chamando a função responsável por achar o melhor corte
        no = self.verifica_melhor_corte(banco)

        # Aqui calculamos o número de ocorrência de cada classe no subconjunto "esquerdo" do banco.
        n_ocorrencias_classes_esquerda = list(Counter(no.filho_esquerdo[self.coluna_alvo]).values())
        n_classes_esquerda = len(list(Counter(no.filho_esquerdo[self.coluna_alvo]).values()))

        # Aqui calculamos o número de ocorrência de cada classe no subconjunto "direito" do banco.
        n_ocorrencias_classes_direita = list(Counter(no.filho_direito[self.coluna_alvo]).values())
        n_classes_direita = len(list(Counter(no.filho_direito[self.coluna_alvo]).values()))

        dominancia_esquerda = 0
        dominancia_direita = 0

        # Aqui é feito o cálculo da classe dominante do subconjunto "esquerdo" do banco.
        for index_classe in range(n_classes_esquerda):
            dominancia_atual = n_ocorrencias_classes_esquerda[index_classe] / sum(n_ocorrencias_classes_esquerda)
            if dominancia_atual > dominancia_esquerda:
                dominancia_esquerda = dominancia_atual

        # Aqui é feito o cálculo da classe dominante do subconjunto "direito" do banco.
        for index_classe in range(n_classes_direita):
            dominancia_atual = n_ocorrencias_classes_direita[index_classe] / sum(n_ocorrencias_classes_direita)
            if dominancia_atual > dominancia_direita:
                dominancia_direita = dominancia_atual

        # Como condição de parada, optamos por uma taxa de 80% de dominância de uma
        # mesma classe no subconjunto do banco, portanto, se isso ocorrer,
        # este subconjunto torna-se um nó folha. Senão, o algoritmo continua
        # e a árvore se divide novamente.
        if dominancia_esquerda < 0.8:
            no.filho_esquerdo = self._cria_arvore_recursiva(no.filho_esquerdo)
        else:
            no.filho_esquerdo = Folha(no.filho_esquerdo, self.coluna_alvo)

        if dominancia_direita < 0.8:
            no.filho_direito = self._cria_arvore_recursiva(no.filho_direito)
        else:
            no.filho_direito = Folha(no.filho_direito, self.coluna_alvo)
            
        return no

    # Neste função verificamos o melhor particionamento do banco,
    # aquele que irá gerar o maior ganho de informação.
    def verifica_melhor_corte(self, banco):
        
        maior_ganho = {'Atributo': '', 'Ganho de Informação': 0, 'Pergunta': None}
        filho_esquerdo = None
        filho_direito = None

        # Cálculo da entropia geral de toda a base de dados.
        entropia_pai = self.calcula_entropia(banco)

        # Para cada atributo e cada valor, dividimos a base de dados em dois,
        # calculamos os ganhos de informação e os armazenamos se for melhor que o anterior.
        for coluna in self.colunas[1:]:

            # Arredondamos os valores, para otimizar o algoritmo.
            banco_coluna = self.arredonda_float(banco[coluna])

            for linha in banco_coluna:
                # Instanciamos a pergunta, de acordo com a divisão atual a ser analisada.
                pergunta = Pergunta(coluna, linha)
                # Temos a base divida pela função, usando a pergunta como parâmetro
                banco_esquerdo, banco_direito = self.corta_banco(banco, pergunta)
                # O ganho de informação é calculado entre os dois subconjuntos de dados
                ganho_info = self.calcula_ganho_informacao(banco_esquerdo, banco_direito, entropia_pai)
                # Se o ganho for melhor que o atual, é armazenado no dicionário
                if ganho_info > maior_ganho['Ganho de Informação']:
                    maior_ganho = {'Atributo': coluna, 'Ganho de Informação': ganho_info, 'Pergunta': pergunta}
                    filho_esquerdo, filho_direito = banco_esquerdo, banco_direito

        # Finalmente, retornamos um nó com todas as informações adquiridas,
        # relacionadas ao maior ganho de informação possível.
        return No(atributo=maior_ganho['Atributo'],
                  entropia=entropia_pai,
                  pergunta=maior_ganho['Pergunta'],
                  filho_esquerdo=filho_esquerdo,
                  filho_direito=filho_direito)

    # Função responsável pelo cálculo de entropia, de acordo com as fórmulas conhecidas.
    def calcula_entropia(self, banco):
        
        entropia = 0
        
        numero_ocorrencias_classe = list(Counter(banco[self.coluna_alvo]).values())

        n_linhas = banco.shape[0]
        
        for quantidade in numero_ocorrencias_classe:
            peso = (quantidade / n_linhas)
            entropia += - peso * math.log(peso, 2)
            
        return entropia

    # Similarmente, é calculado o ganho de informação utilizando os dois subconjuntos
    # da base de dados.
    def calcula_ganho_informacao(self, banco_esquerdo, banco_direito, entropia_pai):
        
        n_amostras = banco_esquerdo.shape[0] + banco_direito.shape[0]
        peso_esquerdo = banco_esquerdo.shape[0] / n_amostras
        peso_direito = banco_direito.shape[0] / n_amostras
        
        if not banco_esquerdo.empty:
            calculo_esquerdo = peso_esquerdo * self.calcula_entropia(banco_esquerdo)
        else:
            calculo_esquerdo = 0
            
        calculo_direito = peso_direito * self.calcula_entropia(banco_direito)
        
        ganho_informacao = calculo_esquerdo + calculo_direito
        
        return entropia_pai - ganho_informacao

    # Função responsável por particionar a base de dados de acordo com a
    # pergunta inserida.
    def corta_banco(self, banco, pergunta):

        # Os dois subconjuntos gerados, cada um com as linhas
        # que deram True para a pergunta, ou False.
        linha_true, linha_false = [], []

        # Iteramos a base de dados e cada linha é avaliada, depois
        # adicionada à lista correspondente.
        for _, row in banco.iterrows():

            if pergunta.verifica(row):
                linha_true.append(row)

            else:
                linha_false.append(row)

        # Conversão das listas de volta para o formato da biblioteca Pandas.
        banco_esquerdo = pd.DataFrame(linha_false)
        banco_direito = pd.DataFrame(linha_true)
        
        return banco_esquerdo, banco_direito

    # Função responsável por arredondar os valores possíveis do banco de dados,
    # no momento da partição, a fim de melhorar o tempo de execução do algoritmo.
    def arredonda_float(self, banco):
        return list(map(int, Counter(['%d' % elem for elem in list(Counter(banco).keys())])))
    
    # Função responsável por percorrer a árvore com o indivíduo a ser classificado.
    # Primeiramente, verifica-se o tipo de nó atual, e se for uma folha, a classe é
    # retornada. Senão, a função entra na recursão percorrendo os nós de acordo com
    # as perguntas de cada um relação ao indivíduo.
    def percorre_arvore(self, linha, no_pai):

        if isinstance(no_pai, Folha):
            return no_pai.classe

        else:
            indice_pergunta = list(self.banco_de_dados.columns).index(no_pai.pergunta.coluna)

            if linha[indice_pergunta] >= no_pai.pergunta.valor:
                return self.percorre_arvore(linha, no_pai.filho_direito)

            else:
                return self.percorre_arvore(linha, no_pai.filho_esquerdo)

    # Função responsável por fazer a classificação do banco inserido,
    # após a construção da árvore.
    def classifica(self, banco):

        serie_predicao = []

        # O banco é iterado e cada linha percorre a árvore
        # a fim de ser classificada, e tal resutaldo é armazenado
        # em uma lista.
        for _, linha in banco.iterrows():
            classe = self.percorre_arvore(linha, self.raiz)
            serie_predicao.append(classe)

        # Após esse processo, a lista é convertida para o objeto Series
        # do Pandas e então concatenado com a coluna alvo do banco de dados.
        serie_predicao = pd.Series(serie_predicao, name='predicao')
        predicao = pd.concat([banco[self.coluna_alvo], serie_predicao], axis=1)

        # Retornamos esse Dataframe contendo a coluna alvo e suas predições
        return predicao

    # Função responsável por imprimir a árvore, percorre a estrutura de forma
    # similar a outras funções já apresentadas, de forma recursiva.
    def imprime(self, no_pai, espacamento=""):

        # Se o nó for folha, imprimimos a sua predição, de acordo com a pergunta.
        if isinstance(no_pai, Folha):
            print(f'{espacamento} Predição: {no_pai.classe}')
            return

        print(f'{espacamento} {str(no_pai.pergunta)}')

        # Imprimimos True, ilustrando as respostas à pergunta anterior
        print(f'{espacamento} --> True:')
        self.imprime(no_pai.filho_direito, espacamento + "  ")

        # Imprimimos False, ilustrando as respostas à pergunta anterior
        print(f'{espacamento} --> False:')
        self.imprime(no_pai.filho_esquerdo, espacamento + "  ")


# Execução da Árvore

Finalmente, temos a execução da árvore, por dez vezes seguidas, a fim de capturar os dados e termos uma melhor média, já que utilizamos alguns processos estocásticos no algoritmo.

In [26]:
banco = bd_vinho
coluna_alvo = 'Vinho'
n_execucoes = 10
verbose = False

acuracias = {'Base Teste': [],'Base Treino': [], 'Base Total': []}

for execucao in range(n_execucoes):
    
    base_treino, base_teste, tipos_saidas = tratar_bd(banco, coluna_alvo)
    dict_bancos = {'Base Teste': base_teste,'Base Treino': base_treino, 'Base Total': banco}
    
    ad = ArvoreDecisao(base_treino, coluna_alvo)
    
    for nome_banco in list(dict_bancos.keys()):

        predicao = ad.classifica(dict_bancos[nome_banco])
        matriz_confusao = np.zeros((3, 3))

        soma = 0

        for _, linha in predicao.iterrows():
            valor_predicao = int(linha[1]) - 1
            valor_real = int(linha[0]) - 1

            matriz_confusao[valor_predicao][valor_real] += 1

        acuracias[nome_banco].append(calcula_resultados(matriz_confusao, verbose))

acuracia_bdteste = f"Desvio Padrão das Acurácias da base de teste: {np.std(acuracias['Base Teste']):.2f} // Média das Acurácias da base de teste: {np.mean(acuracias['Base Teste']):.2f}%"
acuracia_bdtreino = f"Desvio Padrão das Acurácias da base de treino: {np.std(acuracias['Base Treino']):.2f} // Média das Acurácias da base de treino: {np.mean(acuracias['Base Treino']):.2f}%"
acuracia_bdtotal = f"Desvio Padrão das Acurácias da base total: {np.std(acuracias['Base Total']):.2f} // Média das Acurácias da base total: {np.mean(acuracias['Base Total']):.2f}%"

print(acuracia_bdteste)
print(acuracia_bdtreino)
print(acuracia_bdtotal)


Desvio Padrão das Acurácias da base de teste: 5.46 // Média das Acurácias da base de teste: 92.08%
Desvio Padrão das Acurácias da base de treino: 2.41 // Média das Acurácias da base de treino: 93.68%
Desvio Padrão das Acurácias da base total: 2.83 // Média das Acurácias da base total: 93.20%
